## nfl lines machine learning project


source data for historical spreads and game scores
http://www.repole.com/sun4cast/data.html

In [2]:
import os

# define the root directory for the nfl code in $MLNLF_ROOT
codeDir = "".join([os.environ['MLNFL_ROOT'], os.path.sep])
dataRoot = "".join([codeDir, "data", os.path.sep])

os.chdir(codeDir)

print(codeDir)
print(dataRoot)

/Users/amit/repos/mlnfl/nfl/
/Users/amit/repos/mlnfl/nfl/data/


In [3]:
# warnings control
import warnings
# choose default, ignore, always
warnings.filterwarnings('default')


In [4]:
# import necessary modules
%matplotlib inline

from __future__ import division
from __future__ import print_function
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import madden

from sklearn import linear_model
from sklearn import svm
from sklearn import ensemble

print ("pandas version ",pd.__version__)

pandas version  0.16.2


In [5]:
# location of lookup files
from referencedata import ReferenceData
lookupFiles = { 'teams' : {'file': 'nflTeams.csv' },
                'seasons' : {'file': 'seasons.csv' },
               }

lookupDir = "".join([dataRoot, 'lookup', os.path.sep])

print ("lookupFiles = %s" % lookupFiles)
print ("lookupDir = %s" % lookupDir)

lookupFiles = {'seasons': {'file': 'seasons.csv'}, 'teams': {'file': 'nflTeams.csv'}}
lookupDir = /Users/amit/repos/mlnfl/nfl/data/lookup/


In [6]:
# import reference data
reference_data = ReferenceData(lookupDir)
reference_data.teams_df.head()

,city,mascot,league,division,year
team,,,,,
Baltimore Ravens,NaN,Ravens,afc,north,NaN
New England Patriots,NaN,Patriots,afc,east,NaN
Tennessee Titans,NaN,Titans,afc,south,NaN
Atlanta Falcons,NaN,Falcons,nfc,south,NaN
Tampa Bay Buccaneers,NaN,Buccaneers,nfc,south,NaN


In [7]:
###  multi-season training
reload(madden)

# testYears and trainYears need to be arrays
trainYears = [2014] #range(2008,2013) 
testYear = [2015]

# train on previous 3 yrs of data
testYear = [2015]
trainYears = range(testYear[0]-3,testYear[0]) 

# training data set - includes one extra year for prev yr record
seasons = np.array(trainYears)
print ("training seasons >> ", seasons)

training seasons >>  [2012 2013 2014]


In [8]:
reload(madden)

# get training data
# 1 - read all the games
path_to_lines = dataRoot + "lines/"
dfAllGames = madden.readGamesAll(path_to_lines, seasons)
# 2 - compile season record for all teams
dfAllTeams = madden.seasonRecord(dfAllGames, reference_data)
# 3 - apply season records and compute other fields for all games
dfAllGames = madden.processGames(dfAllGames, dfAllTeams, reference_data)
# 4 - remove extra year of data 
dfAllGames = dfAllGames[dfAllGames.season.isin(seasons)]

# use different test set 
seasonTest = np.array(testYear) # should be only one year
print ("results for >> ", seasonTest)
# 1 - read all the games
dfGamesTest = madden.readGamesAll(path_to_lines, seasonTest)  
# 2 - compile season record for all teams
dfTeamsTest = madden.seasonRecord(dfGamesTest,reference_data) 
# 3 - apply season records and compute other fields for all games
dfGamesTest = madden.processGames(dfGamesTest, dfTeamsTest, reference_data) 
# 4 - remove extra year of data 
dfGamesTest = dfGamesTest[dfGamesTest.season.isin(seasonTest)]


madden.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  team_df['gamesPlayed'] = range(1, len(team_df.index) + 1)  # index 1 thur 16
madden.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  team_df['team'] = team
madden.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  team_df['homeGame'] = se

results for >>  [2015]


In [12]:
dfAllGames

,Date,Visitor,Visitor Score,Home Team,Home Score,Line,Total Line,season,finalLine,yahooLine,...,homeWin,favoredWin,gameWeek,homeRecord,visitorRecord,favoredRecord,underdogRecord,prevFavoredRecord,prevUnderdogRecord,absLine
7918,9/5/12,Dallas Cowboys,24,New York Giants,17,4.0,46,2012,NaN,NaN,...,0,0,1,0.000000,0.000000,0.000000,0.000000,0.5625,0.5000,4.0
7919,9/9/12,Indianapolis Colts,21,Chicago Bears,41,10.0,42.5,2012,NaN,NaN,...,1,1,1,0.000000,0.000000,0.000000,0.000000,0.5000,0.1250,10.0
7920,9/9/12,Philadelphia Eagles,17,Cleveland Browns,16,-9.5,42,2012,NaN,NaN,...,0,1,1,0.000000,0.000000,0.000000,0.000000,0.5000,0.2500,9.5
7921,9/9/12,Buffalo Bills,28,New York Jets,48,3.0,39,2012,NaN,NaN,...,1,1,1,0.000000,0.000000,0.000000,0.000000,0.5000,0.3750,3.0
7922,9/9/12,Washington Redskins,40,New Orleans Saints,32,9.0,50.5,2012,NaN,NaN,...,0,0,1,0.000000,0.000000,0.000000,0.000000,0.8125,0.3125,9.0
7923,9/9/12,New England Patriots,34,Tennessee Titans,13,-6.5,48,2012,NaN,NaN,...,0,1,1,0.000000,0.000000,0.000000,0.000000,0.8125,0.5625,6.5
7924,9/9/12,Jacksonville Jaguars,23,Minnesota Vikings,26,3.5,38,2012,NaN,NaN,...,1,1,1,0.000000,0.000000,0.000000,0.000000,0.1875,0.3125,3.5
7925,9/9/12,Miami Dolphins,10,Houston Texans,30,13.5,41.5,2012,NaN,NaN,...,1,1,1,0.000000,0.000000,0.000000,0.000000,0.6250,0.3750,13.5
7926,9/9/12,St Louis Rams,23,Detroit Lions,27,9.0,46.5,2012,NaN,NaN,...,1,1,1,0.000000,0.000000,0.000000,0.000000,0.6250,0.1250,9.0
7927,9/9/12,Atlanta Falcons,40,Kansas City Chiefs,24,-2.5,43,2012,NaN,NaN,...,0,1,1,0.000000,0.000000,0.000000,0.000000,0.6250,0.4375,2.5


In [9]:
# define independent variables for logistic regression
features = ['favoredRecord','underdogRecord',  # current year records of both teams
            'prevFavoredRecord','prevUnderdogRecord', # prev year records, helps early in season when only few games played
            'gameWeek',  # week in season, should make a good/bad record later in season more important
            'absLine',  # absolute value of spread since favored team already determined
            'divisionGame', # T/F, usually more competitive rivalry games, i.e. bad teams still win home division games.
            'favoredHomeGame', # T/F, important since output of classifier is "did the favored team win?"
            ]

# run the classifer
random_state = 11
#classifier = svm.SVC(kernel='poly',probability=True, random_state=random_state)
classifier = linear_model.LogisticRegression(C=1e5)
mlClassifier = madden.runScikitClassifier(dfAllGames,features,classifier)

# apply results of logistic regression to the test set
#dfPredict = madden.predictGames(dfGamesTest,mlClassifier,features)

# apply ranking logic and determine scoring outcomes for league
#dfAll = madden.rankGames(dfPredict,reference_data,seasonTest[0])

In [10]:
# predict one week of current season
iweek = 7
reload(madden)

# use different test set - current year
testYear = [2015]
seasonTest = np.array(testYear) # should be only one year
print ("results for >> ", seasonTest)

dfGamesTest = madden.readGamesAll(path_to_lines, seasonTest)
dfTeamsTest = madden.seasonRecord(dfGamesTest,reference_data)
dfGamesTest = madden.processGames(dfGamesTest, dfTeamsTest, reference_data)
dfGamesTest = dfGamesTest[dfGamesTest.season.isin(seasonTest)]

# pick only this weeks games for predict
dfTest = dfGamesTest[dfGamesTest.gameWeek == iweek]

# apply results of logistic regression to the test set
dfPredict = madden.predictGames(dfTest,mlClassifier,features)
# apply ranking logic and determine scoring outcomes for league
dfAll = madden.rankGames(dfPredict, reference_data, seasonTest[0])



results for >>  [2015]


madden.py:411: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dfPredict['predict_proba'] = pp[:, 1]
madden.py:412: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dfPredict['predict_proba_abs'] = abs(pp[:, 1] - 0.5)
madden.py:418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dfPredict['predictWin'] = 1 * ((s

In [14]:
dfGamesTest[dfGamesTest.gameWeek == iweek]

,Date,Visitor,Visitor Score,Home Team,Home Score,Line,Total Line,season,finalLine,yahooLine,...,homeWin,favoredWin,gameWeek,homeRecord,visitorRecord,favoredRecord,underdogRecord,prevFavoredRecord,prevUnderdogRecord,absLine
8777,10/22/15,Seattle Seahawks,NaN,San Francisco 49ers,NaN,-6.0,NaN,2015,NaN,NaN,...,NaN,NaN,7,0.333333,0.333333,0.333333,0.333333,0.7500,0.5000,6.0
8778,10/25/15,Buffalo Bills,NaN,Jacksonville Jaguars,NaN,-5.5,NaN,2015,NaN,NaN,...,NaN,NaN,7,0.166667,0.500000,0.500000,0.166667,0.5625,0.1875,5.5
8779,10/25/15,Tampa Bay Buccaneers,NaN,Washington Redskins,NaN,3.5,NaN,2015,NaN,NaN,...,NaN,NaN,7,0.333333,0.333333,0.333333,0.333333,0.2500,0.1250,3.5
8780,10/25/15,Atlanta Falcons,NaN,Tennessee Titans,NaN,-4.5,NaN,2015,NaN,NaN,...,NaN,NaN,7,0.166667,0.833333,0.833333,0.166667,0.3750,0.1250,4.5
8781,10/25/15,New Orleans Saints,NaN,Indianapolis Colts,NaN,4.5,NaN,2015,NaN,NaN,...,NaN,NaN,7,0.500000,0.333333,0.500000,0.333333,0.6875,0.4375,4.5
8782,10/25/15,Minnesota Vikings,NaN,Detroit Lions,NaN,-2.5,NaN,2015,NaN,NaN,...,NaN,NaN,7,0.166667,0.500000,0.500000,0.166667,0.4375,0.6875,2.5
8783,10/25/15,Pittsburgh Steelers,NaN,Kansas City Chiefs,NaN,-2.0,NaN,2015,NaN,NaN,...,NaN,NaN,7,0.166667,0.666667,0.666667,0.166667,0.6875,0.5625,2.0
8784,10/25/15,Cleveland Browns,NaN,St Louis Rams,NaN,5.5,NaN,2015,NaN,NaN,...,NaN,NaN,7,0.333333,0.333333,0.333333,0.333333,0.3750,0.4375,5.5
8785,10/25/15,Houston Texans,NaN,Miami Dolphins,NaN,4.5,NaN,2015,NaN,NaN,...,NaN,NaN,7,0.333333,0.333333,0.333333,0.333333,0.5000,0.5625,4.5
8786,10/25/15,New York Jets,NaN,New England Patriots,NaN,9.0,NaN,2015,NaN,NaN,...,NaN,NaN,7,0.833333,0.666667,0.833333,0.666667,0.7500,0.2500,9.0


In [11]:
# display weekly ranking output

# ranking methods choices
# 0. pick based on spread
# 1. always pick favored team, rank by probability of win
# 2. pick winner based on abs(probability - .5), rank by probability
# 3. pick winner based on abs(probability - .5), rank by abs(probability - .5)

dispCols = ['season','gameWeek','Visitor','visitorRecord','Home Team','homeRecord',
            'Line','prevFavoredRecord','prevUnderdogRecord','predict_proba',
            'lineGuess','probaGuess', 'probaAbsGuess', 'predictTeam']


dfAll['predictTeam'] = np.where((dfAll['predict_proba'] - .5) > 0 , dfAll['favorite'], dfAll['underdog'])
guessCol = 'probaGuess'
predictCols = ['gameWeek','predictTeam', 'predict_proba', guessCol, 'favorite','lineGuess', 'Line']
dfAll[predictCols].sort(guessCol, ascending=False)

,gameWeek,predictTeam,predict_proba,probaGuess,favorite,lineGuess,Line
8786,7,New England Patriots,0.838849,16,New England Patriots,16,9.0
8790,7,Arizona Cardinals,0.829496,15,Arizona Cardinals,15,7.5
8788,7,New York Giants,0.693856,14,New York Giants,7,3.5
8780,7,Atlanta Falcons,0.689779,13,Atlanta Falcons,9,-4.5
8777,7,Seattle Seahawks,0.675882,12,Seattle Seahawks,14,-6.0
8789,7,Carolina Panthers,0.668644,11,Carolina Panthers,5,3.0
8781,7,Indianapolis Colts,0.651330,10,Indianapolis Colts,11,4.5
8778,7,Buffalo Bills,0.647091,9,Buffalo Bills,12,-5.5
8784,7,St Louis Rams,0.640078,8,St Louis Rams,13,5.5
8782,7,Minnesota Vikings,0.636101,7,Minnesota Vikings,4,-2.5


In [17]:
week_filter = dfAll.gameWeek == 7
dfAll['predictTeam'] = np.where((dfAll['predict_proba'] - .5) > 0 , dfAll['favorite'], dfAll['underdog'])
dfAll[week_filter][predictCols].sort(guessCol, ascending=False)

,gameWeek,predictTeam,predict_proba,probaGuess,favorite,lineGuess,Line
8786,7,New England Patriots,0.838849,16,New England Patriots,16,9.0
8790,7,Arizona Cardinals,0.829496,15,Arizona Cardinals,15,7.5
8788,7,New York Giants,0.693856,14,New York Giants,7,3.5
8780,7,Atlanta Falcons,0.689779,13,Atlanta Falcons,9,-4.5
8777,7,Seattle Seahawks,0.675882,12,Seattle Seahawks,14,-6.0
8789,7,Carolina Panthers,0.668644,11,Carolina Panthers,5,3.0
8781,7,Indianapolis Colts,0.651330,10,Indianapolis Colts,11,4.5
8778,7,Buffalo Bills,0.647091,9,Buffalo Bills,12,-5.5
8784,7,St Louis Rams,0.640078,8,St Louis Rams,13,5.5
8782,7,Minnesota Vikings,0.636101,7,Minnesota Vikings,4,-2.5


In [18]:
# display weekly ranking output for spread method

# ranking methods choices
# 0. pick based on spread
# 1. always pick favored team, rank by probability of win
# 2. pick winner based on abs(probability - .5), rank by probability
# 3. pick winner based on abs(probability - .5), rank by abs(probability - .5)

predictCols = ['favorite','lineGuess', 'absLine','Line', 'favoredHomeGame', 'divisionGame', 'favoredRecord']

sortCols = ['absLine','favoredHomeGame', 'divisionGame', 'favoredRecord', 'favorite']
dfSpread = dfAll[predictCols].sort(sortCols , ascending=False)
#print(dfSpread.to_csv(sys.stdout,sep=',', index=False))
dfSpread

,favorite,lineGuess,absLine,Line,favoredHomeGame,divisionGame,favoredRecord
8786,New England Patriots,16,9.0,9.0,1,1,0.833333
8790,Arizona Cardinals,15,7.5,7.5,1,0,0.666667
8777,Seattle Seahawks,14,6.0,-6.0,0,1,0.333333
8784,St Louis Rams,13,5.5,5.5,1,0,0.333333
8778,Buffalo Bills,12,5.5,-5.5,0,0,0.500000
8781,Indianapolis Colts,11,4.5,4.5,1,0,0.500000
8785,Miami Dolphins,10,4.5,4.5,1,0,0.333333
8780,Atlanta Falcons,9,4.5,-4.5,0,0,0.833333
8787,San Diego Chargers,8,4.0,4.0,1,1,0.333333
8788,New York Giants,7,3.5,3.5,1,1,0.500000
